# CH14 딥러닝 (Keras / TensorFlow)

업로드된 **CH14 딥러닝.ppt** 내용을 바탕으로,  
딥러닝 개요/절차, **이진 분류(당뇨병)**, **다중 분류(붓꽃)**, **회귀(보스턴 주택)**, 그리고 **[플러스 예제] 당뇨병 진행도 예측(개념/대체 실습)** 흐름을  
👉 **설명 + 실습 예제**로 구성한 노트북입니다. fileciteturn8file2

> ⚠️ 실행 환경에 따라 `tensorflow`가 설치되어 있지 않을 수 있습니다.  
> 설치가 필요하면: `pip install tensorflow` (환경 정책에 따라 제한될 수 있음)


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## 14.1 딥러닝의 개요
- 신경망(neural network): 입력→가중치/합→활성화→출력
- 입력층/은닉층/출력층, 층이 깊어지면(deep) 복잡한 패턴 학습
- 절차: 데이터 준비 → 전처리(결측/스케일링) → 모델 정의 → 컴파일(optimizer/loss/metric) → 학습(fit) → 평가(evaluate) → 학습 곡선 시각화 fileciteturn8file2

In [ ]:
# (권장) 딥러닝 예제는 tensorflow.keras를 사용합니다.
try:
    import tensorflow as tf
    from tensorflow.keras import Sequential
    from tensorflow.keras.layers import Dense
    print("TensorFlow version:", tf.__version__)
except Exception as e:
    print("TensorFlow import 실패:", e)


## 14.2 딥러닝 분류: 당뇨병 발병 예측(이진 분류)
PPT에서는 Kaggle Pima Indians 데이터(`diabetes.csv`)를 사용합니다. fileciteturn8file2

여기서는 **파일이 있으면 그대로 사용**, 없으면 **구조를 연습할 샘플 데이터**로 대체합니다.

In [ ]:
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

csv_name = "diabetes.csv"  # 파일이 있으면 자동 사용

if os.path.exists(csv_name):
    df = pd.read_csv(csv_name)
    print("loaded:", df.shape)
else:
    # 샘플 데이터(학습용)
    rng = np.random.default_rng(42)
    n = 600
    df = pd.DataFrame({
        "Pregnancies": rng.integers(0, 10, n),
        "Glucose": rng.normal(120, 25, n).clip(0),
        "BloodPressure": rng.normal(70, 12, n).clip(0),
        "SkinThickness": rng.normal(20, 8, n).clip(0),
        "Insulin": rng.normal(80, 35, n).clip(0),
        "BMI": rng.normal(30, 6, n).clip(0),
        "DiabetesPedigreeFunction": rng.normal(0.5, 0.2, n).clip(0),
        "Age": rng.integers(21, 70, n)
    })
    # Outcome 생성(대략적)
    logit = (df["Glucose"]*0.02 + df["BMI"]*0.08 + df["Age"]*0.01 - 8.0)
    prob = 1/(1+np.exp(-logit))
    df["Outcome"] = (rng.random(n) < prob).astype(int)
    print("sample:", df.shape)

df.head()


### (1) 전처리: 0이 될 수 없는 열을 NaN으로 바꾸고 dropna (PPT 흐름) fileciteturn8file2

In [ ]:
df2 = df.copy()
columns = ["Glucose", "BloodPressure", "SkinThickness", "Insulin", "BMI"]
for c in columns:
    df2[c] = df2[c].replace(0, np.nan)

df2 = df2.dropna()
df2.shape


### (2) 특성/타깃 분리 + 스케일링 + train/test split fileciteturn8file2

In [ ]:
target = df2.pop("Outcome").values
X = df2.values

X_train, X_test, y_train, y_test = train_test_split(
    X, target, test_size=0.2, random_state=42, stratify=target
)

scaler = StandardScaler()
X_train_s = scaler.fit_transform(X_train)
X_test_s = scaler.transform(X_test)

X_train_s.shape, X_test_s.shape


### (3) 모델 정의/컴파일/학습
- 은닉층: relu
- 이진 분류 출력층: sigmoid
- loss: binary_crossentropy
- optimizer: Adam
- metric: accuracy fileciteturn8file2

In [ ]:
# tensorflow가 없는 환경이면 이 셀은 실행되지 않을 수 있어요.
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

model = Sequential([
    Dense(12, activation="relu", input_dim=X_train_s.shape[1]),
    Dense(8, activation="relu"),
    Dense(1, activation="sigmoid")
])

model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
history = model.fit(
    X_train_s, y_train,
    validation_split=0.2,
    epochs=30,
    batch_size=32,
    verbose=0
)

loss, acc = model.evaluate(X_test_s, y_test, verbose=0)
print("test loss:", loss, "test acc:", acc)


### (4) 학습 곡선 시각화: loss/accuracy fileciteturn8file2

In [ ]:
hist = history.history
plt.plot(hist["loss"], label="loss")
plt.plot(hist["val_loss"], label="val_loss")
plt.legend()
plt.title("Loss")
plt.show()

plt.plot(hist["accuracy"], label="acc")
plt.plot(hist["val_accuracy"], label="val_acc")
plt.legend()
plt.title("Accuracy")
plt.show()


## 14.2 LAB) 딥러닝으로 붓꽃 품종 분류(다중 분류)
- 출력층: softmax
- loss: sparse_categorical_crossentropy
PPT의 LAB 흐름을 sklearn iris 데이터로 재현합니다. fileciteturn8file2

In [ ]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

iris = load_iris()
X = iris.data
y = iris.target

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

model2 = Sequential([
    Dense(16, activation="relu", input_dim=X_train.shape[1]),
    Dense(16, activation="relu"),
    Dense(3, activation="softmax")  # 3-class
])

model2.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
h2 = model2.fit(X_train, y_train, validation_split=0.2, epochs=50, batch_size=16, verbose=0)

loss, acc = model2.evaluate(X_test, y_test, verbose=0)
print("test acc:", acc)


## 14.3 딥러닝 회귀: (대체 실습) 캘리포니아 주택 가격
PPT는 보스턴 주택을 예로 들지만, 최신 환경에서는 보스턴 데이터 로딩이 제한되는 경우가 있어
**캘리포니아 주택**으로 동일한 회귀 흐름(스케일링→Dense→mse/mae)을 실습합니다. fileciteturn8file2

In [ ]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

housing = fetch_california_housing()
X = housing.data
y = housing.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

reg = Sequential([
    Dense(64, activation="relu", input_dim=X_train.shape[1]),
    Dense(64, activation="relu"),
    Dense(1)  # 회귀는 활성화 함수 없음(기본 선형)
])

reg.compile(optimizer="adam", loss="mse", metrics=["mae"])
h3 = reg.fit(X_train, y_train, validation_split=0.2, epochs=30, batch_size=32, verbose=0)

loss, mae = reg.evaluate(X_test, y_test, verbose=0)
print("test MSE:", loss, "test MAE:", mae)


## 14.4 [플러스 예제] 당뇨병 진행도 예측
PPT에는 당뇨병 진행도 예측 예제가 포함됩니다. fileciteturn8file2

여기서는 즉시 실행 가능한 **sklearn diabetes(회귀)** 데이터로
딥러닝 회귀 모델을 만들어 예측하는 방식으로 대체 실습합니다.

In [ ]:
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

diab = load_diabetes()
X = diab.data
y = diab.target  # 진행도(연속값)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

dl = Sequential([
    Dense(64, activation="relu", input_dim=X_train.shape[1]),
    Dense(32, activation="relu"),
    Dense(1)
])

dl.compile(optimizer="adam", loss="mse", metrics=["mae"])
h4 = dl.fit(X_train, y_train, validation_split=0.2, epochs=50, batch_size=32, verbose=0)

loss, mae = dl.evaluate(X_test, y_test, verbose=0)
print("test MSE:", loss, "test MAE:", mae)


## 정리 체크리스트
- [ ] 딥러닝(신경망) 구조와 절차를 설명할 수 있다.
- [ ] 이진 분류에서 sigmoid + binary_crossentropy를 사용할 수 있다.
- [ ] 다중 분류에서 softmax + (sparse_)categorical_crossentropy를 사용할 수 있다.
- [ ] 회귀에서 mse/mae를 이용해 학습/평가할 수 있다.
- [ ] 학습 곡선(loss/accuracy)을 해석할 수 있다.
